# Action Menu - Bob

## Role - Responder

In this notebook we'll be going through the Action Menu Protocol. For details on the protocol, please refer to the [RFC](https://github.com/hyperledger/aries-rfcs/tree/master/features/0509-action-menu). Specifically, we'll demonstrate how Bob acts as a Responder to respond to Action Menu related requests from Alice.

A requirement for this protocol to work is an active DIDComm communication channel between Alice and Bob. To achieve that, you should have completed the [Establishing a Connection](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial.

## 1. Initialise Bob controller

We begin by initialising the controller for Bob.

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 2. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending name='Task-4' coro=<AriesAgentController.listen_webhooks() running at /aries_basic_controller/aries_controller.py:61>>

In [4]:
def get_menu_handler(payload):
    print(f"Get Active Menu Webhook: {payload}")

'''
when a menu action is performed, the handler should contain business
logic to handling and executing that action specifically.
'''
def perform_menu_handler(payload):
    print("Perform Menu Action Webhook: ", payload)
    # custom business logic here

get_menu_listener = {
    "topic": "get-active-menu",
    "handler": get_menu_handler
}

perform_menu_listener = {
    "topic": "perform-menu-action",
    "handler": perform_menu_handler
}

agent_controller.register_listeners([get_menu_listener, perform_menu_listener], defaults=True)

Subscribing too: get-active-menu
Subscribing too: perform-menu-action


## 3. Check the agent has an active connection

An active connection between Alice and Bob is required. You can either:

- complete the [Establishing a Connection](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial, or
- running the python script `create_connection.py` in the setup folder

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")

Results :  [{'created_at': '2023-01-01 15:38:32.965206Z', 'invitation_key': 'C9VVc5HS8ASbZGJ4RRH38huh7jxzV1Lxc6dv9MMhYRm6', 'state': 'active', 'request_id': '5ef4cc1f-c5f0-4a09-8bf0-49631cd28b22', 'their_did': '78JbknZS4Un2nRsSXthrrX', 'my_did': '6UJzVcKRUTNd2Z3fkgSwqc', 'rfc23_state': 'completed', 'updated_at': '2023-01-01 15:38:46.632967Z', 'their_role': 'inviter', 'accept': 'manual', 'invitation_mode': 'once', 'connection_id': '9777a4c5-00c9-4816-acd3-3139431cee8b', 'routing_state': 'none', 'their_label': 'Alice'}]
Connection : {'created_at': '2023-01-01 15:38:32.965206Z', 'invitation_key': 'C9VVc5HS8ASbZGJ4RRH38huh7jxzV1Lxc6dv9MMhYRm6', 'state': 'active', 'request_id': '5ef4cc1f-c5f0-4a09-8bf0-49631cd28b22', 'their_did': '78JbknZS4Un2nRsSXthrrX', 'my_did': '6UJzVcKRUTNd2Z3fkgSwqc', 'rfc23_state': 'completed', 'updated_at': '2023-01-01 15:38:46.632967Z', 'their_role': 'inviter', 'accept': 'manual', 'invitation_mode': 'once', 'connection_id': '9777a4c5-00c9-4816-acd3-3139431cee8b', '

## 6. Send Action Menu to Alice

Bob responds to Alice's request by sending over the action menu that Bob supports.

In [6]:
# feel free to edit the menu options.
menu_options = [
      {
        "name": "faber_college_actions",
        "title": "Faber College Actions",
        "description": "Faber College Actions",
        "disabled": "False",
        "form": {
          "title": "Actions",
          "description": "Faber College Actions",
          "params": [{
              "name": "remove_cred_exchange_record",
              "title": "Remove Credential Exchange Record",
              "type": "boolean",
              "required": False
          }, {
              "name": "begin_issue_cred",
              "title": "Begin Issue Credential",
              "type": "boolean",
              "required": False
          }],
          "submit-label": "Send"
        }
      }
    ]

await agent_controller.action_menu.send_menu(
    connection_id=connection_id,
    menu_description="Menu for performing actions with Faber College",
    menu_errormsg="Error: something went wrong",
    menu_title="Faber College Action Menu",
    menu_options=menu_options
)

ClientResponseError: 404, message='Not Found', url=URL('http://bob-agent:8051/connections/9777a4c5-00c9-4816-acd3-3139431cee8b/send-menu')

Error during POST /connections/9777a4c5-00c9-4816-acd3-3139431cee8b/send-menu: 404, message='Not Found', url=URL('http://bob-agent:8051/connections/9777a4c5-00c9-4816-acd3-3139431cee8b/send-menu')


## 7. Continue with step 8 of [Alice's notebook](http://localhost:8889/lab/tree/5%20Advanced%20Concepts/Part%201%20-%20Action%20Menu.ipynb)

## End of Tutorial

Be sure to terminate the controller before running another tutorial.

In [7]:
response = await agent_controller.terminate()
print(response)

None
